In [67]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

import requests

# Method 1: discounted cash flow calculation
The idea is https://www.streetofwalls.com/finance-training-courses/investment-banking-technical-training/discounted-cash-flow-analysis/ You can get the free cash per share, and cash flow growth rate on https://www.gurufocus.com, for example, here is cash flow information for Apple stock: https://www.gurufocus.com/stock/AAPL/dcf. WACC rate is here https://www.gurufocus.com/term/wacc/AAPL/WACC-Percentage

In [68]:
def calculate_intrinsic_value_based_on_discounted_cash_flow(
        current_free_cash_flow_per_share,
        free_cash_flow_per_share_growth_rate,
        projected_number_of_years,
        discount_rate,
        perpetual_growth_rate
    ):
        """
        Args:
            current_free_cash_per_share: Current free cash flow per share.
            free_cash_flow_per_share_growth_rate: The average free cash per share growth rate
                of the company for the last x years.
            projected_number_of_years: Typically, a target’s FCF is projected out 5 to 10 years in the future.
            discount_rate:  The rate of return used to determine the
                present value of future cash flows. Corporations often use the
                Weighted Average Cost of Capital (WACC).
            perpetual_growth_rate: The constant rate that a company is expected to
                grow at forever. Usually choose inflation rate.

        Returns:
            The calculated intrinsic value.
        """

        # Calculate discounted cash flow of the next x years.
        discounted_cash_flow = 0
        for year in range(1, projected_number_of_years + 1):
            cash_flow_at_the_year = current_free_cash_flow_per_share * (
                pow((1 + free_cash_flow_per_share_growth_rate), year)
            )
            discounted_cash_flow += cash_flow_at_the_year / (pow((1 + discount_rate), year))

        # Use Perpetuity Method to calculate the terminal value at x-th year.
        cash_flow_of_last_year = current_free_cash_flow_per_share * (
            pow((1 + free_cash_flow_per_share_growth_rate), projected_number_of_years)
        )
        terminal_value = (
            cash_flow_of_last_year
            * (1 + perpetual_growth_rate)
            / (discount_rate - perpetual_growth_rate)
        )
        discounted_terminal_value = terminal_value / pow(
            (1 + free_cash_flow_per_share_growth_rate), projected_number_of_years
        )

        current_intrinsic_value = discounted_cash_flow + discounted_terminal_value
        return current_intrinsic_value

In [69]:
# Define the function to retrieve cash flow data
def get_cash_flow(api_key, symbol):
    base_url = 'https://www.alphavantage.co/query'
    function = 'CASH_FLOW'
    outputsize='compact'

    # Make the API request
    params = {
        'function': function,
        'symbol': symbol,
        'outputsize': outputsize,
        'apikey': api_key,
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Extract the last data point from the time series
        last_5y = data['annualReports'][:5]
        return last_5y
    else:
        return None

In [70]:
# Define the function to retrieve company data
def get_company_overview(api_key, symbol):
    base_url = 'https://www.alphavantage.co/query'
    function = 'OVERVIEW'
    outputsize='compact'

    # Make the API request
    params = {
        'function': function,
        'symbol': symbol,
        'outputsize': outputsize,
        'apikey': api_key,
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return None

In [71]:
# Define the function to retrieve inflation data
def get_inflation(api_key, symbol):
    base_url = 'https://www.alphavantage.co/query'
    function = 'INFLATION'
    outputsize='compact'

    # Make the API request
    params = {
        'function': function,
        'symbol': symbol,
        'outputsize': outputsize,
        'apikey': api_key,
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Extract the last data point from the time series
        inf = data['data'][:5]
        return inf
    else:
        return None

In [72]:
# Get the current year
current_year = datetime.today().year

# Replace 'YOUR_API_KEY' with your actual Alpha Vantage API key
api_key = '03775CCKZ199ZUG2'

# Define the symbol or stock you want to analyze
ticker = 'ON'  # Replace with the stock symbol you are interested in

# Call the function to get cash flow data
cahs_flow = get_cash_flow(api_key, ticker)

# Call the function to get company overview data
company_overview = get_company_overview(api_key, ticker)

# Call the function to get inflation data
inflation = get_inflation(api_key, ticker)

In [73]:
# Current free cash flow per share

# Calculate free cash flow
current_free_cash_flow = float(cahs_flow[0]['operatingCashflow']) - float(cahs_flow[0]['capitalExpenditures'])

# Replace with the number of shares outstanding for the company
shares_outstanding = float(company_overview['SharesOutstanding'])

# Calculate free cash flow per share
current_free_cash_flow_per_share = current_free_cash_flow / shares_outstanding

current_free_cash_flow_per_share

3.7801429307774823

In [74]:
# Free cash flow per share growth rate for last year
free_cash_flow_last1 = float(cahs_flow[1]['operatingCashflow']) - float(cahs_flow[1]['capitalExpenditures'])

# Free cash flow per share growth rate for the year befor last year
free_cash_flow_last2 = float(cahs_flow[2]['operatingCashflow']) - float(cahs_flow[2]['capitalExpenditures'])

# Replace with the number of shares outstanding for the company
shares_outstanding = float(company_overview['SharesOutstanding'])

free_cash_flow_per_share_last1 = free_cash_flow_last1 / shares_outstanding
free_cash_flow_per_share_last2 = free_cash_flow_last2 / shares_outstanding

free_cash_flow_per_share_growth_rate_lag1 = (current_free_cash_flow_per_share / free_cash_flow_per_share_last1) - 1
free_cash_flow_per_share_growth_rate_lag2 = (free_cash_flow_per_share_last1 / free_cash_flow_per_share_last2) - 1

free_cash_flow_per_share_growth_rate = (free_cash_flow_per_share_growth_rate_lag1*0.7) + (free_cash_flow_per_share_growth_rate_lag2*0.3)

free_cash_flow_per_share_growth_rate

0.6534715866158842

In [75]:
free_cash_flow_per_share_growth_rate

0.6534715866158842

In [76]:
# Projected_number_of_years
projected_number_of_years = 3

In [77]:
# Discount rate (WACC - Weighted Average Cost of Capital)

#A. Equity

treasury_yield10 = yf.Ticker("^TNX")
risk_free_rate = round(treasury_yield10.history(period="1d")["Close"][0] / 100, 4)

# Define the start and end date for your calculation
start_date = datetime.today() - timedelta(days=3*365) # Replace with your desired start date
end_date = datetime.today()    # Replace with your desired end date

# Download S&P 500 Total Return Index data
spx_tri_data = yf.download("^GSPC ", start=start_date, end=end_date, progress=False)

# Calculate the total return
initial_value = spx_tri_data["Close"].iloc[0]
final_value = spx_tri_data["Close"].iloc[-1]
total_return = (final_value / initial_value) - 1

# Display the annual return
sp500_teturn  = total_return / projected_number_of_years


stock = yf.Ticker(ticker)
beta = stock.info["beta"]

cost_of_equity = round(risk_free_rate + beta*(sp500_teturn - risk_free_rate),4)

stock_bal = stock.balance_sheet
weight_of_equity = stock_bal.loc["Stockholders Equity"][0]/ stock_bal.loc["Total Assets"][0]

#B. Debt

# get stock financial statement
stock_Fin = stock.financials

# Cost of Debt
cost_of_debt = round(stock_Fin.loc["Interest Expense"][0] *-1 / stock_bal.loc["Total Debt"][0],4)

weight_of_debt = round(stock_bal.loc["Total Debt"][0]/ stock_bal.loc["Total Assets"][0],4)

#C. Tax Shield
# Tax Rate
tax_rate = round(stock_Fin.loc["Tax Provision"][0] / stock_Fin.loc["Pretax Income"][0],4)

#D. WACC
WACC = round((weight_of_equity * cost_of_equity) + ((weight_of_debt * cost_of_debt ) * (1-tax_rate)),4)

discount_rate = WACC

discount_rate

0.0607

In [78]:
# Perpetual growth rate
#perpetual_growth_rate = float(inflation[0].get('value'))*0.01

perpetual_growth_rate = 0.03 # 2023 US inflartion data is missing, input manually

perpetual_growth_rate

0.03

In [79]:
# Calculate intrinsic value
intrinsic_value_dcf = calculate_intrinsic_value_based_on_discounted_cash_flow(
        current_free_cash_flow_per_share,
        free_cash_flow_per_share_growth_rate,
        projected_number_of_years,
        discount_rate,
        perpetual_growth_rate)

# Print the result
print("{} stock's intrinsic value at the end of {} based on DCF is {:.2f}".format(ticker, current_year + projected_number_of_years, intrinsic_value_dcf))

ON stock's intrinsic value at the end of 2026 based on DCF is 156.22
